In [39]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [40]:
df1 = pd.read_csv('./dataset_appota/game-transaction-data-sample.csv', parse_dates = ['created_date', 'first_login_date'])
df1.head()

,created_date,transaction_id,transaction_type,transaction_amount,user_id,first_login_date
0,2023-06-10,GDP23061032292866A,apple,9693.75000,66438777,2023-06-08
1,2023-06-11,GDP23061132297778A,apple,9693.75000,66438777,2023-06-08
2,2023-06-08,GDP23060832281857A,apple,19485.41667,66438777,2023-06-08
3,2023-06-09,GDP23060932290365A,apple,19485.41667,66438777,2023-06-08
4,2023-06-10,GDP23061032296255G,google,9693.75000,66496542,2023-06-10


In [41]:
#Change type of UserID
df1['user_id'] = df1['user_id'].astype(str)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802 entries, 0 to 801
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   created_date        802 non-null    datetime64[ns]
 1   transaction_id      802 non-null    object        
 2   transaction_type    802 non-null    object        
 3   transaction_amount  802 non-null    float64       
 4   user_id             802 non-null    object        
 5   first_login_date    802 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 37.7+ KB


In [42]:
df1.isna().sum()

created_date          0
transaction_id        0
transaction_type      0
transaction_amount    0
user_id               0
first_login_date      0
dtype: int64

In [43]:
df_duplicate_check = df1[df1.duplicated(subset=['created_date', 'transaction_id', 'transaction_type', 'transaction_amount', 'user_id'], keep=False)]
df_duplicate_check

,created_date,transaction_id,transaction_type,transaction_amount,user_id,first_login_date
275,2023-06-14,GDP23061432314246B,bank,125000.00000,173204,2022-07-01
295,2023-06-14,GDP23061432314246B,bank,125000.00000,173204,2022-07-01
299,2023-06-24,GDP23062432380398B,bank,41666.66667,173204,2022-07-01
318,2023-06-24,GDP23062432380398B,bank,41666.66667,173204,2022-07-01


In [44]:
#case 1: dataset not cleaned
df1= df1.drop_duplicates(subset=['created_date', 'transaction_id', 'transaction_type', 'transaction_amount', 'user_id'], keep='first')

#double check
df1[df1.duplicated(subset=['created_date', 'transaction_id', 'transaction_type', 'transaction_amount', 'user_id'], keep=False)]


,created_date,transaction_id,transaction_type,transaction_amount,user_id,first_login_date


In [45]:
df2 = pd.read_csv('./dataset_appota/logins-data-sample.csv', parse_dates = ['created_date', 'first_login_date'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25548 entries, 0 to 25547
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   created_date      25548 non-null  datetime64[ns]
 1   user_id           25548 non-null  int64         
 2   first_login_date  25548 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1)
memory usage: 598.9 KB


In [46]:
#change type of UserID
df2['user_id'] = df2['user_id'].astype(str)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25548 entries, 0 to 25547
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   created_date      25548 non-null  datetime64[ns]
 1   user_id           25548 non-null  object        
 2   first_login_date  25548 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 598.9+ KB


In [47]:
df2.isna().sum()

created_date        0
user_id             0
first_login_date    0
dtype: int64

# Analytics

#### 1. Daily Active User (DAU)

In [48]:
dau_df = df2.sort_values(['created_date'], ascending = True).drop_duplicates(['created_date','user_id'], keep='first')
dau = dau_df.groupby(['created_date'])[['user_id']].value_counts().reset_index().rename(columns={0: 'count'})
dau = dau.groupby(['created_date'])[['count']].sum().sort_values(['created_date'], ascending=True).reset_index()
dau

,created_date,count
0,2023-06-01,297
1,2023-06-02,306
2,2023-06-03,607
3,2023-06-04,705
4,2023-06-05,306
5,2023-06-06,326
6,2023-06-07,474
7,2023-06-08,307
8,2023-06-09,524
9,2023-06-10,884


In [49]:
fig_dau = px.bar(x=dau['created_date'], y = dau['count'], text=dau['count']) 
fig_dau.update_layout(title = 'Daily Active Users (DAU)', xaxis=dict(title='Date', tickmode='linear', tickangle = -90), yaxis_title = 'No. of Users')
fig_dau.update_traces(textposition='outside')
fig_dau.show()

Users are mostly active on weekends.

#### 2. Daily New User

In [50]:
dnu_df = df2[df2['created_date'] == df2['first_login_date']]
dnu_df = dnu_df.sort_values(['created_date'], ascending = True).drop_duplicates(subset=['created_date','user_id','first_login_date'], keep='first')
dnu = dnu_df.groupby(['first_login_date'])[['user_id']].value_counts().reset_index().rename(columns={0: 'count'}).groupby(['first_login_date'])[['count']].sum().reset_index()
dnu_df

,created_date,user_id,first_login_date
4620,2023-06-01,66419800,2023-06-01
15452,2023-06-01,66421017,2023-06-01
9176,2023-06-01,66419386,2023-06-01
2083,2023-06-01,66420580,2023-06-01
22493,2023-06-01,66420852,2023-06-01
...,...,...,...
9768,2023-06-30,66565581,2023-06-30
23854,2023-06-30,66566258,2023-06-30
20066,2023-06-30,66565428,2023-06-30
23270,2023-06-30,66566110,2023-06-30


In [51]:
fig_dnu = px.bar(x=dnu['first_login_date'], y = dnu['count'], text=dnu['count'])
fig_dnu.update_layout(title = 'Daily New Users (DNU)', xaxis=dict(title='First Login Date', tickmode='linear', tickangle = -90), yaxis_title = 'No. of Users')
fig_dnu.update_traces(textposition='outside')
fig_dnu.show()

#### 3. Retention Rate (D1, D7) (only for references)

In [52]:
#d1
# step 1: check all the user ID in D0
# step 2: check all the user ID in D1 whether it matches that of in D0, yes --> +1, no --> bỏ
# step 3: check until day 30

#d2
#d3

#average D1, d2, d3, d7


#### 4. Average Revenue Per User (ARPU)

In [53]:
df1_arpu = df1.groupby(['created_date'])['transaction_amount'].sum().reset_index()
df1_arpu['transaction_amount'] = round(df1_arpu['transaction_amount'], 2)
df1_arpu


,created_date,transaction_amount
0,2023-06-01,2584939.58
1,2023-06-02,1300025.00
2,2023-06-03,4005441.67
3,2023-06-04,1302304.17
4,2023-06-05,3583052.08
5,2023-06-06,1484706.25
6,2023-06-07,1831287.50
7,2023-06-08,1771789.58
8,2023-06-09,3468493.75
9,2023-06-10,3887770.83


In [54]:
#daily all revenue per DAU
arpu = pd.merge(df1_arpu, dau)
arpu['ARPU'] = round(arpu['transaction_amount'] / arpu['count'], 2)
arpu

,created_date,transaction_amount,count,ARPU
0,2023-06-01,2584939.58,297,8703.50
1,2023-06-02,1300025.00,306,4248.45
2,2023-06-03,4005441.67,607,6598.75
3,2023-06-04,1302304.17,705,1847.24
4,2023-06-05,3583052.08,306,11709.32
5,2023-06-06,1484706.25,326,4554.31
6,2023-06-07,1831287.50,474,3863.48
7,2023-06-08,1771789.58,307,5771.30
8,2023-06-09,3468493.75,524,6619.26
9,2023-06-10,3887770.83,884,4397.93


In [55]:
fig_arpu = px.bar(x=arpu['created_date'], y = arpu['ARPU'], text=arpu['ARPU'])
fig_arpu.update_layout(title = 'Daily Average Revenue Per DAU', xaxis=dict(title='Date', tickmode='linear', tickangle = -90), yaxis_title = 'VND')
fig_arpu.update_traces(textposition='outside')
fig_arpu.show()

#### 5. IAP Revenue per Paying Users (ARPPU)

In [56]:
#Net IAP revenue / Paying Users (can be calculate in daily, weekly)
arppu_df = df1.sort_values(['created_date'], ascending = True).drop_duplicates(subset=['created_date', 'user_id']).groupby(['created_date'])[['user_id']].value_counts().reset_index()
arppu_df=arppu_df.groupby(['created_date'])[['count']].sum().reset_index()
arppu_df


,created_date,count
0,2023-06-01,12
1,2023-06-02,7
2,2023-06-03,16
3,2023-06-04,6
4,2023-06-05,8
5,2023-06-06,6
6,2023-06-07,7
7,2023-06-08,8
8,2023-06-09,15
9,2023-06-10,21


In [57]:
arppu = pd.merge(df1_arpu, arppu_df)
arppu

,created_date,transaction_amount,count
0,2023-06-01,2584939.58,12
1,2023-06-02,1300025.00,7
2,2023-06-03,4005441.67,16
3,2023-06-04,1302304.17,6
4,2023-06-05,3583052.08,8
5,2023-06-06,1484706.25,6
6,2023-06-07,1831287.50,7
7,2023-06-08,1771789.58,8
8,2023-06-09,3468493.75,15
9,2023-06-10,3887770.83,21


In [58]:
arppu['ARPPU'] = round(arppu['transaction_amount']/arppu['count'],2)
arppu

,created_date,transaction_amount,count,ARPPU
0,2023-06-01,2584939.58,12,215411.63
1,2023-06-02,1300025.00,7,185717.86
2,2023-06-03,4005441.67,16,250340.10
3,2023-06-04,1302304.17,6,217050.69
4,2023-06-05,3583052.08,8,447881.51
5,2023-06-06,1484706.25,6,247451.04
6,2023-06-07,1831287.50,7,261612.50
7,2023-06-08,1771789.58,8,221473.70
8,2023-06-09,3468493.75,15,231232.92
9,2023-06-10,3887770.83,21,185131.94


In [59]:
fig_arppu = px.bar(x=arppu['created_date'], y = arppu['ARPPU'], text=arppu['ARPPU'])
fig_arppu.update_layout(title = 'Daily Average Revenue Per Paying Users', xaxis=dict(title='Date', tickmode='linear', tickangle = -90), yaxis_title = 'VND')
fig_arppu.update_traces(textposition='inside', textangle=-90)
fig_arppu.show()

#### 6. IAP Revenue per Transactions (ARPT)

In [60]:
# Net IAP Revenue / Transactions (daily)
arpt_df = df1.groupby(['created_date'])[['transaction_id']].value_counts().reset_index()
arpt_df= arpt_df.groupby(['created_date'])[['count']].sum().reset_index()
arpt_df

,created_date,count
0,2023-06-01,25
1,2023-06-02,12
2,2023-06-03,63
3,2023-06-04,8
4,2023-06-05,21
5,2023-06-06,12
6,2023-06-07,13
7,2023-06-08,13
8,2023-06-09,29
9,2023-06-10,88


In [61]:
arpt = pd.merge(df1_arpu, arpt_df)
arpt['ARPT'] = round(arpt['transaction_amount']/arpt['count'],2)

In [62]:
fig_arpt = px.bar(x=arpt['created_date'], y = arpt['ARPT'], text=arpt['ARPT'])
fig_arpt.update_layout(title = 'Daily Average Revenue Per Transaction', xaxis=dict(title='Date', tickmode='linear', tickangle = -90), yaxis_title = 'VND')
fig_arpt.update_traces(textposition='inside', textangle=-90)

fig_arpt.show()

#### 7. Paying Users Rate (PUR)

In [63]:
# Paying Users/ active users (daily)
pur_df = pd.merge(arppu_df, dau, left_index=True, right_index=True)
pur_df['PUR'] = round((pur_df['count_x'] / pur_df['count_y'])*100, 2)
pur_df

,created_date_x,count_x,created_date_y,count_y,PUR
0,2023-06-01,12,2023-06-01,297,4.04
1,2023-06-02,7,2023-06-02,306,2.29
2,2023-06-03,16,2023-06-03,607,2.64
3,2023-06-04,6,2023-06-04,705,0.85
4,2023-06-05,8,2023-06-05,306,2.61
5,2023-06-06,6,2023-06-06,326,1.84
6,2023-06-07,7,2023-06-07,474,1.48
7,2023-06-08,8,2023-06-08,307,2.61
8,2023-06-09,15,2023-06-09,524,2.86
9,2023-06-10,21,2023-06-10,884,2.38


In [64]:
fig_pur = px.line(x=pur_df['created_date_x'], y = pur_df['PUR'], text=[f'{i}%' for i in pur_df['PUR']])
fig_pur.update_layout(title = 'Paying User Rate (Daily)', xaxis=dict(title='Date', tickmode='linear', tickangle = -90), yaxis = dict(title = '%', ticksuffix="%"))
fig_pur.update_traces(textposition='top center')
fig_pur.show()

#### 8. New Paying Users

In [65]:
# users who send their first IAP pack purchased event that day (only count in this month as we do not have enough data for the previous month)
npu_df = df1.sort_values(['created_date'], ascending = True).drop_duplicates(['user_id'], keep='first').groupby(['created_date'])[['user_id']].value_counts().reset_index()
npu = npu_df.groupby(['created_date'])[['count']].sum().reset_index()
npu

,created_date,count
0,2023-06-01,12
1,2023-06-03,7
2,2023-06-04,2
3,2023-06-05,2
4,2023-06-06,2
5,2023-06-07,2
6,2023-06-08,1
7,2023-06-09,6
8,2023-06-10,9
9,2023-06-11,3


In [66]:
fig_npu = px.bar(x=npu['created_date'], y = npu['count'], text=npu['count'])
fig_npu.update_layout(title = 'Daily New Paying User', xaxis=dict(title='Date', tickmode='linear', tickangle = -90), yaxis = dict(title = 'Users'))
fig_npu.update_traces(textposition='outside')
fig_npu.show()

#### 9. Paying Users 

In [67]:
# Number of users made purchase (daily)
fig_arppu_df = px.bar(x=arppu_df['created_date'], y = arppu_df['count'], text=arppu_df['count'])
fig_arppu_df.update_layout(title = 'Daily Paying User', xaxis=dict(title='Date', tickmode='linear', tickangle = -90), yaxis = dict(title = 'Users'))
fig_arppu_df.update_traces(textposition='outside')
fig_arppu_df.show()

#### 10. Total transaction

In [68]:
transaction = round(df1['transaction_amount'].sum(), 2)
transaction

80893472.91

#### 11. Whales active

In [69]:
# active whales = those users that have spend in our game mor than 100 EURO lifetime and they were also active in the target day/month. For ex, if a user is spending 100EURO in sept and  in oct is active but spend 0 euro, we are going to count him in both months.
#Top paying users

pu = df1.groupby(['user_id'])[['transaction_amount']].sum().sort_values(['transaction_amount'], ascending = False).reset_index()
pu['transaction_amount'] = round(pu['transaction_amount'], 2)
whale_pu = pu[pu['transaction_amount'] > 2767920]
dolphin_pu = pu[(pu['transaction_amount'] > 53548) & (pu['transaction_amount'] <= 2768920)]
minnow_pu = pu[(pu['transaction_amount'] > 0) & (pu['transaction_amount'] <= 53548)]
dolphin_pu

,user_id,transaction_amount
8,65258151,2583333.33
9,66397659,2399081.25
10,62141790,2083333.33
11,65949513,1083333.33
12,66174123,941666.67
13,61056100,916666.67
14,57845561,804412.50
15,62503345,750000.00
16,19479600,645833.33
17,62973580,645833.33


In [70]:
whale_list = whale_pu['user_id'].astype(str).tolist()
whale_active = df2[df2['user_id'].isin(whale_list)]
whale_active = whale_active.groupby('created_date')[['user_id']].value_counts().reset_index()
whale_active


,created_date,user_id,count
0,2023-06-01,173204,5
1,2023-06-01,63431300,5
2,2023-06-01,33368153,4
3,2023-06-01,66087401,4
4,2023-06-01,66174545,4
...,...,...,...
231,2023-06-30,33368153,3
232,2023-06-30,63431300,2
233,2023-06-30,66170983,2
234,2023-06-30,66174632,2


In [71]:
# Number of users made purchase (daily)
fig_whale_active = px.bar(x=whale_active['created_date'], y = whale_active['count'], color = whale_active['user_id'])
fig_whale_active.update_layout(title = 'Daily Active Whale', xaxis=dict(title='Date', tickmode='linear', tickangle = -90), yaxis = dict(title = 'No. of login'))
fig_whale_active.show()

#### 12. Top Whales Revenue

In [77]:
total = df1['transaction_amount'].sum()
total

80893472.91295001

#### 13. Whales average revenue per whales

In [72]:
average_whale = whale_pu['transaction_amount'].sum()/8
average_whale

7605341.1475

#### 14. Whales active paying

In [73]:
# Whales that were active and buy an IAP in the same day or month

In [74]:
#test retention


In [75]:
dates = df2.sort_values(['created_date'], ascending=True).drop_duplicates(subset=['created_date'], keep='first')['created_date'].tolist()
user_lists = [test[test['created_date'] == date]['user_id'].tolist() for date in dates]

array = []
for i in range(len(user_lists)):
    if i == 0:
        array.append(len(user_lists[i]))  # Compare list1 with list1
    else:
        array.append(len([x for x in user_lists[i] if x in user_lists[i-1]]))

array

NameError: name 'test' is not defined

In [ ]:
data = {'created_date': dates, 'retention_number': array}
df = pd.DataFrame(data)
df['DAU'] = dau['count']
df['retention_rate'] = 0


for i in range(30):
    if i == 0: 
        df['retention_rate'][i] = round((df['retention_number'][i]/ df['DAU'][i])*100,2)
    else:
        df['retention_rate'][i] = round((df['retention_number'][i]/ df['DAU'][i-1])*100,2)
    
df

C:\Users\aki\AppData\Local\Temp\ipykernel_17236\3796323640.py:9: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


C:\Users\aki\AppData\Local\Temp\ipykernel_17236\3796323640.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy o

,created_date,retention_number,DAU,retention_rate
0,2023-06-01,297,297,100.00
1,2023-06-02,206,306,69.36
2,2023-06-03,236,607,77.12
3,2023-06-04,417,705,68.70
4,2023-06-05,248,306,35.18
5,2023-06-06,192,326,62.75
6,2023-06-07,240,474,73.62
7,2023-06-08,219,307,46.20
8,2023-06-09,199,524,64.82
9,2023-06-10,338,884,64.50


In [ ]:
fig_df = px.line(x=df['created_date'], y = df['retention_rate'],text=[f'{i}%' for i in df['retention_rate']])
fig_df.update_layout(title = 'Retention Rate', xaxis=dict(title='Date', tickmode='linear', tickangle = -90), yaxis = dict(title = 'Retention Rate', ticksuffix="%"))
fig_df.update_traces(textposition='top center')
fig_df.show()

In [ ]:
dnu_df
user_lists_test = [dnu_df[dnu_df['created_date'] == date]['user_id'].tolist() for date in dates]

array1 = []
for i in range(len(user_lists_test)):
    if i == 0:
        array1.append(len(user_lists_test[i]))  # Compare list1 with list1
    else:
        array1.append(len([x for x in user_lists[i] if x in user_lists_test[i-1]]))
        
array1


[20,
 5,
 22,
 16,
 6,
 5,
 39,
 7,
 4,
 61,
 55,
 10,
 4,
 7,
 50,
 5,
 62,
 50,
 19,
 65,
 7,
 34,
 30,
 32,
 5,
 3,
 9,
 10,
 7,
 6]

In [ ]:
data1 = {'created_date': dates, 'retention_number_DNU': array1}
df_dnu_retention = pd.DataFrame(data1)
df_dnu_retention['DNU'] = dnu['count']
df_dnu_retention['retention_rate_DNU'] = 0


for i in range(30):
    if i == 0: 
        df_dnu_retention['retention_rate_DNU'][i] = round((df_dnu_retention['retention_number_DNU'][i]/ df_dnu_retention['DNU'][i])*100,2)
    else:
        df_dnu_retention['retention_rate_DNU'][i] = round((df_dnu_retention['retention_number_DNU'][i]/ df_dnu_retention['DNU'][i-1])*100,2)
    
df_dnu_retention

C:\Users\aki\AppData\Local\Temp\ipykernel_17236\2664010960.py:9: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


C:\Users\aki\AppData\Local\Temp\ipykernel_17236\2664010960.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy o

,created_date,retention_number_DNU,DNU,retention_rate_DNU
0,2023-06-01,20,20,100.00
1,2023-06-02,5,34,25.00
2,2023-06-03,22,61,64.71
3,2023-06-04,16,93,26.23
4,2023-06-05,6,17,6.45
5,2023-06-06,5,59,29.41
6,2023-06-07,39,37,66.10
7,2023-06-08,7,35,18.92
8,2023-06-09,4,185,11.43
9,2023-06-10,61,171,32.97


In [ ]:
df_retention = df.merge(df_dnu_retention, how='left')
df_retention

,created_date,retention_number,DAU,retention_rate,retention_number_DNU,DNU,retention_rate_DNU
0,2023-06-01,297,297,100.00,20,20,100.00
1,2023-06-02,206,306,69.36,5,34,25.00
2,2023-06-03,236,607,77.12,22,61,64.71
3,2023-06-04,417,705,68.70,16,93,26.23
4,2023-06-05,248,306,35.18,6,17,6.45
5,2023-06-06,192,326,62.75,5,59,29.41
6,2023-06-07,240,474,73.62,39,37,66.10
7,2023-06-08,219,307,46.20,7,35,18.92
8,2023-06-09,199,524,64.82,4,185,11.43
9,2023-06-10,338,884,64.50,61,171,32.97


In [ ]:
fig_df_retention = px.line(x=df_retention['created_date'], y = [df_retention['retention_rate'],df_retention['retention_rate_DNU']])
fig_df_retention.update_layout(title = 'Retention Rate', xaxis=dict(title='Date', tickmode='linear', tickangle = -90), yaxis = dict(title = 'Retention Rate', ticksuffix="%"))
fig_df_retention.update_traces(textposition='top center')
fig_df_retention.show()

Note: 
- Minnow (> 0 EURO and  <= 20 EURO)
- Dolphin (>20 and <= 100)
- Whale (>100 and <= 1000)
- Kraken (> 1000 and <= 1000000)